Testing various AIs with various lookup-table "value" functions on tic-tac-toe

In [1]:
import numpy as np 

Hashes out all possible board trajectories, then grades them.
Working backwards from all finished states (with either 0, 0.5, or 1 grade), for any non-finished
state, assigns maximum value of all possible future moves if my turn, minimum value if
opponent's turn.

The function assumes that it's "symbol"'s turn.

Speed: O(n!) where n is # of slots -> ~O(3^n) with dynamic programming to make a full lookup table

Simplicity: idk, took ~an hour for me to code up

In [11]:
utilities = {}
def utility(board: np.array, symbol, opp_symbol) -> int:
    #print(board)
    init_score = grade_board(board)
    if init_score != -1:
        return init_score
    
    all_traj = []
    for row in range(3):
        for col in range(3):
            if board[row][col] == 0:
                board_copy = np.copy(board)
                board_copy[row][col] = symbol
                all_traj.append(board_copy)
    
    grades = []
    for traj in all_traj:
        key = (traj.tobytes(), opp_symbol)
        if not (key in utilities.keys()):
            utilities[key] = utility(traj, opp_symbol, symbol)
        grades.append(utilities[key])
    # grades = [utility(traj, opp_symbol, symbol) for traj in all_traj]
    #print(all_traj)
    #print(grades)
    if symbol in grades:
        return symbol
    if 0.5 in grades:
        return 0.5
    return opp_symbol

"""
Grades a board -- either the winner, 0.5 if draw, or -1 if not finished
(given one player is 1, the other is 2)
Inspiration: https://gist.github.com/qianguigui1104/edb3b11b33c78e5894aad7908c773353 
"""
def grade_board(board: np.array):
    for row in range(3):
        if board[row][0] == board[row][1] == board[row][2] and board[row][0] != 0:
            return board[row][0]
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] and board[0][col] != 0:
            return board[0][col]
        
    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != 0:
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != 0:
        return board[0][2]
    
    full = True
    for row in range(3):
        for col in range(3):
            if board[row][col] == 0:
                full = False
    if full:
        return 0.5
    return -1

def hash(board, symbol) -> int:
    out = 0
    primes = [2, 3, 5, 7, 11, 13, 17, 19, 23]
    for row in range(3):
        for col in range(3):
            out += primes[3*row + col] * board[row][col]
    out += 29 * symbol
    return

print(utility(np.zeros((3, 3)), 1, 2)) #expected: 0.5
print(len(utilities))
print(utility(np.array([[1, 0, 1], [2, 2, 0], [1, 0, 2]]), 1, 2)) 
#X _ X | O O _ | X _ O; expected: 1
print(utility(np.array([[1, 0, 0], [0, 2, 0], [0, 0, 0]]), 1, 2))
#X _ _ | _ O _ | _ _ _; expected: 0.5

0.5
5477
1
0.5


Is this a utility function model or a value model? The recursion part might make it a value model,
but we're also effectively using brute-force here (the only "heuristic" here is end-game score).

What would a reward/value lookup table for tic-tac-toe look like? 
- If there's two-in-a-row of our symbol and the other spot is blank, max reward (we can win on the next turn)
- If there's two-in-a-row of their symbol and the other spot is blank, all moves besides filling that blank spot have min reward
- If we have the opportunity to create a "fork" (a setup where we have two symbols in two different row/column/diagonals, making it impossible for the opponent to stop us next turn from winning), max reward
- Otherwise, maybe having zero opp_symbol and >= 1 symbol in a row/column/diagonal is positive reward?
    - Maybe having a symbol in a corner or the center is good
    - These don't seem very meaningful/indicative of anything, and are harder to code up

e.g. see https://github.com/SamarpanCoder2002/Tic-Tac-Toe-AI/blob/main/Tic-Tac-Toe-AI.py for a heuristics AI (~500 lines!)

Heuristics: see https://blog.ostermiller.org/tic-tac-toe-strategy/ for an example

Notable points:
- Value/Reward looks simliar to utility in returning a number, but doesn't do recursion
- In return, grade_board has to be more complicated (act() is simpler than utility())

In [16]:
#Value/Reward -- O(n) * (runtime of grade_board) where n = # cells
def act(board, symbol, opp_symbol) -> tuple:
    init_score = grade_board(board, symbol, opp_symbol)
    if init_score == 0 or init_score == 1: #game is over
        return init_score
    
    all_traj = []
    moves = []
    for row in range(3):
        for col in range(3):
            if board[row][col] == 0:
                board_copy = np.copy(board)
                board_copy[row][col] = symbol
                all_traj.append(board_copy)
                moves.append((row, col))
    
    grades = [grade_board(traj, opp_symbol, symbol) for traj in all_traj]
    #grades contains reward/value for each possible move
    if symbol in grades:
        return symbol
    return moves[grades.index(min(grades))] #tuple resulting in the worst position for the opponent

#Calculates reward/value
def grade_board(board: np.array, symbol, opp_symbol):
    """
    Pseudocode:
    
    """
    return np.random.rand()
print(act(np.zeros((3, 3)), 1, 2))

(1, 2)


Claim: there exists a structure w/n NN (e.g. some Python function) that can be described as one of these non-degenerately (where non-degenerate <-> predictive OOD)

ex: chess
- Utility: {all possible set of moves} -> {0, 0.5, 1}
- Reward: -3 pts if lose knight, etc.
- Value: rating the “total value” of the current board (e.g. by summing up all the values of the pieces; this would produce a pretty bad chess AI, but not that bad)
- Heuristics: e.g. if Sicilian, play 5… a6

In this case, the code becomes more "complicated", harder to write and harder to interpret as we sacrifice more speed. If we change the game a little bit (e.g. make it so that each player can place 1s or 2s; the first player to get three-in-a-row of either symbol wins), the heuristics will also fail to generalize, whereas a small modification to the grade_board and/or utility function will result in the utility function working. (You do have to carefully modify grade_board, though; I guess in some sense, we use the heuristic that we are in a 3x3 board, only 1s and 2s are played, etc.)

In [ ]:
#Heuristics
def act_heuristics(board: np.array, symbol, opp_symbol) -> tuple:
    """
    Example pseudocode:
    if (\exists two symbols in a line) and (third space is blank):
        return third space
    if (\exists space with one symbol and one blank in two lines sharing that space):
        return space  #can create a fork
    return np.random.rand(board.shape)
    """